In [21]:
import os
print('os.getcwd()', os.getcwd())
import sys
sys.path.insert(1, '../')
print(sys.version)
import time

#plotting tools
from matplotlib import pyplot as plt 
from tqdm.notebook import tqdm as tqdm

#torch libs
import torch
print('torch.__version__', torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe_device = 0 if torch.cuda.is_available() else -1

#huggingface transformers
import transformers
print('transformers.__version__',transformers.__version__)
from transformers import AutoTokenizer, pipeline

from datasets import load_dataset

#curious
from curious.models import GPT2HeadWithValueModel
from curious.rl import PPOTrainer
from curious.utils import LengthSampler, collater, respond_to_batch

#jupyter stuff
%load_ext autoreload
%autoreload 2

os.getcwd() /Users/carson/projects/language_reinforce/notebooks
3.10.6 (main, Aug 30 2022, 05:09:33) [Clang 12.0.0 (clang-1200.0.32.29)]
torch.__version__ 1.12.1
transformers.__version__ 4.22.2
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
cache_folder = '/Users/carson/projects/modelstates' #'/home/carson/Desktop/Projects/modelstates'
model_name = 'distilgpt2'#'gpt2' # 'gpt2-xl' # 'gpt2-large' #
cache_dir = os.path.join(cache_folder, model_name)
cache_dir

'/Users/carson/projects/modelstates/distilgpt2'

In [4]:
gpt2_model = GPT2HeadWithValueModel.from_pretrained(
    model_name,
    cache_dir=cache_dir
)
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained(
    model_name,
    cache_dir=cache_dir
)

Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['transformer.h.5.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'v_head.summary.weight', 'transformer.h.4.attn.masked_bias', 'v_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['transformer.h.5.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'v_head.summary.weight', 'transformer.h.4.attn.masked_bias', 'v_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infer

```
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.10.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.3.attn.masked_bias', 'lm_head.weight', 'h.9.attn.masked_bias', 'h.6.attn.masked_bias', 'h.11.attn.masked_bias', 'h.5.attn.masked_bias', 'v_head.summary.weight', 'h.4.attn.masked_bias', 'h.1.attn.masked_bias', 'h.0.attn.masked_bias', 'h.2.attn.masked_bias', 'v_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
```

In [5]:
# load imdb with datasets
ds = load_dataset('imdb', split='train')
ds = ds.rename_columns({'text': 'review', 'label': 'sentiment'})
ds = ds.filter(lambda x: len(x["review"])>200, batched=False)

Found cached dataset imdb (/Users/carson/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Loading cached processed dataset at /Users/carson/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-e68378636d846987.arrow


In [6]:
# load the reward model used to assign reward to model's outputs
sentiment_pipe = pipeline("sentiment-analysis","lvwerra/distilbert-imdb", device=pipe_device)

In [7]:
# reward model configs
sent_kwargs = {
    "return_all_scores": True,
    "function_to_apply": "none",
    "batch_size": 16
}

text = 'this movie was really bad!!'
sentiment_pipe(text, **sent_kwargs)

/Users/carson/projects/language_reinforce/venv/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  warnings.warn(


[[{'label': 'NEGATIVE', 'score': 2.335047721862793},
  {'label': 'POSITIVE', 'score': -2.7265758514404297}]]

In [8]:
# load the tokenizer

gpt2_tokenizer = AutoTokenizer.from_pretrained(model_name)
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

In [9]:
# modify the dataset with a new tokens column and query column
# these are used in the data feeder of the main train loop

input_size = LengthSampler(min_value = 4, max_value = 8)
output_size = LengthSampler(min_value = 4, max_value = 16)

def map_tokenize(sample):
    
    '''
    this function is applied to the dataset and 
    only the first few tokens of review are used for "tokens"
    they are decoded and stored as query in their text form
    '''
    
    sample["tokens"] = gpt2_tokenizer.encode(sample["review"])[:input_size()]
    sample["query"] = gpt2_tokenizer.decode(sample["tokens"])
    
    return sample

ds = ds.map(map_tokenize, batched=False)
dataloader = torch.utils.data.DataLoader(ds, batch_size=32, collate_fn=collater)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/24895 [00:00<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


In [11]:
config = {
    "model_name": "lvwerra/gpt2-imdb",
    "cls_model_name": "lvwerra/distilbert-imdb",
    "steps": 20000,
    "batch_size": 32, 
    "forward_batch_size": 16,
    "ppo_epochs": 4,   
    "txt_in_min_len": 4,
    "txt_in_max_len": 8,
    "txt_out_min_len": 4,
    "txt_out_max_len": 16,
    "lr": 1e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
}

In [ ]:
# instantiate the reinforcement learning trainer
reward_list = []

ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, gpt2_tokenizer, **config)

total_ppo_epochs = 150
batch_size = 32

for epoch, batch in tqdm(zip(range(total_ppo_epochs), iter(dataloader)), total = total_ppo_epochs):
    
    ### initiate logging dicts
    logs, timing = dict(), dict()
    t0 = time.time()
    
    ### get start sequence input tokens
    query_tensors = [torch.tensor(t).long().to(device) for t in batch["tokens"]]
    
    ### Get responses from policy model
    t = time.time()
    response_tensors = []
    for i in range(batch_size):
        gen_len = output_size()
        query_tensor = query_tensors[i].unsqueeze(dim=0)
        response_tensor = respond_to_batch(gpt2_model, query_tensor, gen_len)
        response_tensors.append(response_tensor.squeeze())
        
    batch['response'] = [gpt2_tokenizer.decode(r.squeeze()) for r in response_tensors]
    timing['time/get_response'] = time.time()-t
    
    #### Compute sentiment score (reward)
    t = time.time()
    texts = [q + r for q,r in zip(batch['query'], batch['response'])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = torch.tensor([output[1]["score"] for output in pipe_outputs]).to(device)
    timing['time/get_sentiment_preds'] = time.time()-t
    
    reward_list.append(torch.mean(rewards).item())
    
    #### Run PPO step 
    t = time.time()
    train_stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    timing['time/optimization'] = time.time()-t
    
plt.figure()
plt.xlabel('Epochs', fontsize=25)
plt.ylabel('Rewards', fontsize=25)
plt.plot(reward_list, label='sentiment')
plt.legend()
plt.show()

  0%|          | 0/150 [00:00<?, ?it/s]